In [1]:
import re
import os
import json
import time
import argparse
import pickle as pkl
import os.path as op
import networkx as nx
from tqdm import tqdm
from copy import deepcopy

In [2]:
class CatIndex():
    
    @classmethod
    def empty(C):
        g = nx.Graph()
        last = "None"
        return(C(g, last))
    
    @staticmethod
    def load(fn):
        with open(fn, "rb") as ih:
            return(pkl.load(ih))
        
    def __init__(self, g, last):
        self.g = g
        self.last = last
    
    @staticmethod
    def examine(s):
        explicits = re.findall("\[\[\w+\]\]", s)
        explicits = [a.replace("[[", "").replace("]]", "") for a in explicits]
        s = {
            "explicits": explicits
        }
        return(s)
    
    def save(self, fn):
        with open(fn, "wb") as oh:
            pkl.dump(self, oh)
    
    def index(self, list_of_files):
        lst = sorted(
            deepcopy(list_of_files), 
            key=lambda x: int(op.split(x)[-1].replace(".md", ""))
        )
        if self.last != "None":
            lst = lst[lst.index(self.last):]
        for a in tqdm(list_of_files):
            with open(a, "r") as ih:
                a_file = ih.read()
            a_linkables = CatIndex.examine(a_file)
            a_cid = op.split(a)[-1].replace(".md", "")
            for b in lst:
                if a != b:
                    with open(b, "r") as ih:
                        b_file = ih.read()
                    b_linkables = CatIndex.examine(b_file)
                    ebunch = {}
                    add_edge = False
                    for c in b_linkables:
                        ints = list(
                            set(a_linkables[c]).intersection(set(b_linkables[c]))
                        )
                        if len(ints) > 0:
                            add_edge = True
                            ebunch[c] = ",".join(ints)
                        else:
                            add_edge = False
                    if add_edge:
                        b_cid = op.split(b)[-1].replace(".md", "")
                        self.g.add_edges_from([(a_cid, b_cid, ebunch)])

In [3]:
I = CatIndex.empty()

In [5]:
files = [a for a in os.walk("/home/bakirillov/exocortex/")][0][2]
files = [op.join("/home/bakirillov/exocortex", a) for a in files]

In [6]:
I.index(files)

100%|██████████| 4/4 [00:00<00:00, 3812.14it/s]


In [4]:
u = [a for a in I.g.edges]

In [5]:
u

[('28052020082348', '28052020224804')]

In [6]:
I.g.edges["28052020082348", "28052020224804"]

{'explicits': 'сегментация,segmentation'}

In [13]:
I.save("test.pkl")

In [3]:
I = CatIndex.load("test.pkl")